# Implementaremos o Naive Bayes para ter uma base de acurácia

Importando as bibliotecas necessárias

In [114]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


Pegando a base de dados de treino

In [115]:
train = pd.read_csv(r"C:\Users\thiag\OneDrive\Desktop\datamining\data\train.csv")

In [116]:
train.shape

(20758, 18)

Pegando a base de dados de teste

In [117]:
test = pd.read_csv(r"C:\Users\thiag\OneDrive\Desktop\datamining\data\train.csv")

In [118]:
train.head(5)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [119]:
test.head(5)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


A coluna id é desnecessária

In [120]:
train = train.drop("id", axis=1)
test = test.drop("id", axis=1)

Testando se tem algum dado vazio

In [121]:
NAs = pd.concat([train.isnull().sum()], axis=1, keys=["Train"])
NAs[NAs.sum(axis=1)>0]

,Train


Nenhum dado vazio

Vendo os tipos de dado de cada coluna

In [122]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          20758 non-null  object 
 1   Age                             20758 non-null  float64
 2   Height                          20758 non-null  float64
 3   Weight                          20758 non-null  float64
 4   family_history_with_overweight  20758 non-null  object 
 5   FAVC                            20758 non-null  object 
 6   FCVC                            20758 non-null  float64
 7   NCP                             20758 non-null  float64
 8   CAEC                            20758 non-null  object 
 9   SMOKE                           20758 non-null  object 
 10  CH2O                            20758 non-null  float64
 11  SCC                             20758 non-null  object 
 12  FAF                             

Todos com tipo 'object' tem que ser convertidos

Vendo a quantidade de dados unicos que cada objeto tem

In [123]:
train.nunique()

Gender                               2
Age                               1703
Height                            1833
Weight                            1979
family_history_with_overweight       2
FAVC                                 2
FCVC                               934
NCP                                689
CAEC                                 4
SMOKE                                2
CH2O                              1506
SCC                                  2
FAF                               1360
TUE                               1297
CALC                                 3
MTRANS                               5
NObeyesdad                           7
dtype: int64

In [124]:
test.nunique()

Gender                               2
Age                               1703
Height                            1833
Weight                            1979
family_history_with_overweight       2
FAVC                                 2
FCVC                               934
NCP                                689
CAEC                                 4
SMOKE                                2
CH2O                              1506
SCC                                  2
FAF                               1360
TUE                               1297
CALC                                 3
MTRANS                               5
NObeyesdad                           7
dtype: int64

Para objetos com mais de 2 dados unicos faremos um dicionario

In [125]:
categorical_columns = train.select_dtypes(include= ['object'])
categorical_columns.columns

Index(['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
       'SCC', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

Esses tem que ser convertidos

In [126]:
train['NObeyesdad'].unique()

array(['Overweight_Level_II', 'Normal_Weight', 'Insufficient_Weight',
       'Obesity_Type_III', 'Obesity_Type_II', 'Overweight_Level_I',
       'Obesity_Type_I'], dtype=object)

dicionario com 7 entradas

In [127]:
target = {'Insufficient_Weight' : '0',
          'Normal_Weight' : '1',
          'Overweight_Level_I' : '2',
          'Overweight_Level_II' : '3',
          'Obesity_Type_I' : '4',
          'Obesity_Type_II' : '5',
          'Obesity_Type_III' : '6',
        }

In [128]:
train['NObeyesdad'] = train['NObeyesdad'].replace(target).astype('int64')
train['NObeyesdad'].unique()

array([3, 1, 0, 6, 5, 2, 4], dtype=int64)

repetir o procedimento acima com as outras colunas

In [129]:
train['MTRANS'].unique()

array(['Public_Transportation', 'Automobile', 'Walking', 'Motorbike',
       'Bike'], dtype=object)

In [130]:
target = {'Walking' : '0',
          'Bike' : '1',
          'Motorbike' : '2',
          'Public_Transportation' : '3',
          'Automobile' : '4',
          }

In [131]:
train['MTRANS'] = train['MTRANS'].replace(target).astype('int64')
train['MTRANS'].unique()

array([3, 4, 0, 2, 1], dtype=int64)

In [132]:
test['MTRANS'] = test['MTRANS'].replace(target).astype('int64')

In [133]:
train['CAEC'].unique()

array(['Sometimes', 'Frequently', 'no', 'Always'], dtype=object)

In [134]:
train['CALC'].unique()

array(['Sometimes', 'no', 'Frequently'], dtype=object)

In [135]:
target = {'no' : '0',
          'Sometimes' : '1',
          'Frequently' : '2',
          'Always' : '3',
          }

In [136]:
train['CAEC'] = train['CAEC'].replace(target).astype('int64')
train['CAEC'].unique()

array([1, 2, 0, 3], dtype=int64)

In [137]:
test['CAEC'] = test['CAEC'].replace(target).astype('int64')

In [138]:
target = {'no' : '0',
          'Sometimes' : '1',
          'Frequently' : '2',
          }

In [139]:
train['CALC'] = train['CALC'].replace(target).astype('int64')
train['CALC'].unique()

array([1, 0, 2], dtype=int64)

In [141]:
test['CALC'] = test['CALC'].replace(target).astype('int64')
test['CALC'].unique()

array([1, 0, 2], dtype=int64)